In [1]:
from selenium import webdriver
from selenium.common import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager  # To download Edge WebDriver
from selenium.common.exceptions import NoSuchElementException
from itertools import chain
import pandas as pd
import os
import time
import numpy as np

In [2]:
# Set up the Edge WebDriver
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service)  # Use the Edge WebDriver

# click on year to get table function, get url of each year
ori_url = "https://www.boxofficemojo.com/year/world/?ref_=bo_nb_di_tab"
def getURL(ori_url):
    driver.get(ori_url)
    driver.implicitly_wait(5)
    open_list = driver.find_element(By.XPATH, '//*[@id="a-autoid-0-announce"]')
    open_list.click()
    list_year_tag = driver.find_element(By.XPATH, '//*[@id="a-popover-1"]')
    driver.execute_script("arguments[0].click();", list_year_tag)
    listYear = driver.find_element(By.XPATH, '//*[@id="a-popover-1"]/div/div/ul')
    list = listYear.find_elements(By.TAG_NAME, "li")
    dict = {}
    for li in list:
        links = li.find_elements(By.TAG_NAME, "a")
        for link in links:
            urlTemp = 'https://www.boxofficemojo.com/year/world/'
            dict.update({li.text: urlTemp + li.text})
    driver.quit()
    return dict   
listOfYear = getURL(ori_url)

In [3]:
listOfYear = dict(list(listOfYear.items())[0:10])

In [4]:
listOfYear

{'2024': 'https://www.boxofficemojo.com/year/world/2024',
 '2023': 'https://www.boxofficemojo.com/year/world/2023',
 '2022': 'https://www.boxofficemojo.com/year/world/2022',
 '2021': 'https://www.boxofficemojo.com/year/world/2021',
 '2020': 'https://www.boxofficemojo.com/year/world/2020',
 '2019': 'https://www.boxofficemojo.com/year/world/2019',
 '2018': 'https://www.boxofficemojo.com/year/world/2018',
 '2017': 'https://www.boxofficemojo.com/year/world/2017',
 '2016': 'https://www.boxofficemojo.com/year/world/2016',
 '2015': 'https://www.boxofficemojo.com/year/world/2015'}

In [5]:
# get table in each year
def getTable(url, csv_file_path):
    service = Service(EdgeChromiumDriverManager().install())
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDriver
    driver.get(url)
    driver.implicitly_wait(15)
    year = driver.find_element(By.XPATH, '//*[@id="a-autoid-0-announce"]/span')
    tbody = driver.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody')
    raw_data = []

    for tr in tbody.find_elements(By.XPATH, '//tr'):
        row = [item.text for item in tr.find_elements(By.XPATH, ".//td")]
        raw_data.append(row)
    data = pd.DataFrame(raw_data)
    # drop 2 first row 
    data.drop([0, 1], inplace=True)
    # remove index
    data.reset_index(drop=True, inplace=True)
    # rename column
    data.columns = ['Rank' ,'Release Group', 'Worldwide', 'Domestic', '%', 'Foreign', '%']
    # add column year
    data['Year'] = year.text
    # convert from string to int 
    data['Year'] = data['Year'].astype(int)
    # convert from dataframe to csv file
    data.to_csv(csv_file_path, mode='a', index=False, sep=',', encoding='utf-8')
    driver.quit() # close browser
    return data
# testData = getTable(ori_url, file_path)

In [11]:
current_directory = os.getcwd() # get current directory //D:\VSC\selenium
ori_url = "https://www.boxofficemojo.com/year/world/?ref_=bo_nb_di_tab"
file_path = os.path.join(current_directory, 'data.csv')
testData = getTable(listOfYear['2022'], file_path)

In [12]:
testData

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%,Year
0,1,Avatar: The Way of Water,"$2,320,250,281","$684,075,767",29.5%,"$1,636,174,514",70.5%,2022
1,2,Top Gun: Maverick,"$1,495,696,292","$718,732,821",48.1%,"$776,963,471",51.9%,2022
2,3,Jurassic World Dominion,"$1,001,978,080","$376,851,080",37.6%,"$625,127,000",62.4%,2022
3,4,Doctor Strange in the Multiverse of Madness,"$955,775,804","$411,331,607",43%,"$544,444,197",57%,2022
4,5,Minions: The Rise of Gru,"$940,203,765","$370,270,765",39.4%,"$569,933,000",60.6%,2022
...,...,...,...,...,...,...,...,...
196,197,Listy do M. 5,"$7,328,061",-,-,"$7,328,061",100%,2022
197,198,Laid-Back Camp Movie,"$7,317,913",-,-,"$7,317,913",100%,2022
198,199,Osomatsusan the Movie 2022 Re-release,"$7,297,522",-,-,"$7,297,522",100%,2022
199,200,The New Toy,"$7,295,051",-,-,"$7,295,051",100%,2022


In [14]:
def getData():
    for value in listOfYear.values():
        edge_options = Options()
        edge_options.add_argument('--disable-blink-features=AutomationControlled')
        driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDrivers
        getTable(value, file_path)
        driver.quit()
        time.sleep(10)
getData()

KeyboardInterrupt: 

In [82]:
def getText(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.text
    except NoSuchElementException:
        return np.nan

In [85]:
error_url = 'https://www.boxofficemojo.com/title/tt30476486/?ref_=bo_gr_ti'
service = Service(EdgeChromiumDriverManager().install())
edge_options = Options()
edge_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDrivers
driver.get(error_url)
print(getText('//*[@id="a-page"]/main/div/div[3]/div[4]/div[2]/span[2]/a/span'))
driver.quit()

nan


In [6]:
'''
Click Steps:
1. Click on movie's name
2. Click on Title summary
'''
def getInfo(url):
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDriver
    driver.get(url)
    driver.implicitly_wait(5)
    count = 1
    try:
        driver.find_element(By.XPATH, '//*[@id="title-summary-refiner"]/a').click() # click on Title Summary
        tempList = []
        try:
            producer = driver.find_element(By.XPATH, ' //*[@id="a-page"]/main/div/div[3]/div[4]/div[1]/span[2]') # get value of Domestic Distributor
            name = producer.text.split('\n')
            count = count + 1
            tempList.append(name[0])
        except NoSuchElementException:
            tempList.append(np.nan)
        def get_text(xpath):
            try:
                element = driver.find_element(By.XPATH, xpath)
                return element.text
            except NoSuchElementException:
                return np.nan
        tempList.append(get_text('//*[@id="a-page"]/main/div/div[3]/div[4]/div['+ count +']/span[2]/a/span')) # get Domestic Opening
        tempList.append(get_text('//*[@id="a-page"]/main/div/div[3]/div[4]/div['+ count +']/span[2]')) # get Earliest release date
        tempList.append(get_text('//*[@id="a-page"]/main/div/div[3]/div[4]/div['+ count +']/span[2]')) # get MPAA
        tempList.append(get_text('//*[@id="a-page"]/main/div/div[3]/div[4]/div['+ count +']/span[2]')) # get Running time
        tempList.append(get_text('//*[@id="a-page"]/main/div/div[3]/div[4]/div['+ count +']/span[2]')) # get Genres
    except NoSuchElementException:
        return [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    driver.quit()
    return tempList

In [89]:
testURL = 'https://www.boxofficemojo.com/title/tt15239678/?ref_=bo_gr_ti'
testInfo = getInfo(testURL)

In [90]:
testInfo

['Warner Bros',
 '$82,505,391',
 'February 28, 2024 (EMEA, APAC)',
 'PG-13',
 '2 hr 46 min',
 'Action Adventure Drama Sci-Fi']

In [93]:
again= getInfo('https://www.boxofficemojo.com/title/tt30476486/?ref_=bo_gr_ti')
again

['Crunchyroll',
 nan,
 '1 hr 25 min',
 'Animation Comedy Drama Sport',
 'See more details at IMDbPro',
 nan]

In [119]:
again1 = getInfo('https://www.boxofficemojo.com/releasegroup/gr4002566661/?ref_=bo_ydw_table_37')
again1

[nan, nan, nan, nan, nan, nan]

In [122]:
again2 = getInfo('https://www.boxofficemojo.com/title/tt27075304/?ref_=bo_gr_ti')
again2

['March 28, 2024 (Germany)',
 nan,
 'Comedy Fantasy',
 'See more details at IMDbPro',
 nan,
 nan]

In [69]:
def getLinks(url):
    service = Service(EdgeChromiumDriverManager().install())
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDriver
    driver.get(url)
    driver.implicitly_wait(10)
    tbody = driver.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody')
    movies = []
    count = 2
    for tr in tbody.find_elements(By.XPATH, '//tr'):
        link = tr.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody/tr['+ str(count) +']/td[2]/a').get_attribute('href')
        count += 1
        if (count == 201):
            break
        #  //*[@id="table"]/div/table[2]/tbody/tr[3]/td[2]/a
        movies.append(link)
    driver.quit()
    return movies

In [70]:
getLinks('https://www.boxofficemojo.com/year/world/?ref_=bo_nb_tt_tab')

['https://www.boxofficemojo.com/releasegroup/gr2058375685/?ref_=bo_ydw_table_1',
 'https://www.boxofficemojo.com/releasegroup/gr2995671557/?ref_=bo_ydw_table_2',
 'https://www.boxofficemojo.com/releasegroup/gr361583109/?ref_=bo_ydw_table_3',
 'https://www.boxofficemojo.com/releasegroup/gr77484549/?ref_=bo_ydw_table_4',
 'https://www.boxofficemojo.com/releasegroup/gr3314176517/?ref_=bo_ydw_table_5',
 'https://www.boxofficemojo.com/releasegroup/gr4020130309/?ref_=bo_ydw_table_6',
 'https://www.boxofficemojo.com/releasegroup/gr127685125/?ref_=bo_ydw_table_7',
 'https://www.boxofficemojo.com/releasegroup/gr2055819781/?ref_=bo_ydw_table_8',
 'https://www.boxofficemojo.com/releasegroup/gr916214277/?ref_=bo_ydw_table_9',
 'https://www.boxofficemojo.com/releasegroup/gr3499381253/?ref_=bo_ydw_table_10',
 'https://www.boxofficemojo.com/releasegroup/gr3197587973/?ref_=bo_ydw_table_11',
 'https://www.boxofficemojo.com/releasegroup/gr831869445/?ref_=bo_ydw_table_12',
 'https://www.boxofficemojo.com

In [104]:
def getAllLinks(listOfYear):
    movies = []
    for year in listOfYear.values():
        movies.append(getAllLink(year))
    movies = list(np.concatenate(movies).flat)
    return movies

In [105]:
info = getAllLinks(listOfYear)

In [106]:
info

['https://www.boxofficemojo.com/releasegroup/gr2058375685/?ref_=bo_ydw_table_1',
 'https://www.boxofficemojo.com/releasegroup/gr2995671557/?ref_=bo_ydw_table_2',
 'https://www.boxofficemojo.com/releasegroup/gr361583109/?ref_=bo_ydw_table_3',
 'https://www.boxofficemojo.com/releasegroup/gr77484549/?ref_=bo_ydw_table_4',
 'https://www.boxofficemojo.com/releasegroup/gr3314176517/?ref_=bo_ydw_table_5',
 'https://www.boxofficemojo.com/releasegroup/gr4020130309/?ref_=bo_ydw_table_6',
 'https://www.boxofficemojo.com/releasegroup/gr127685125/?ref_=bo_ydw_table_7',
 'https://www.boxofficemojo.com/releasegroup/gr2055819781/?ref_=bo_ydw_table_8',
 'https://www.boxofficemojo.com/releasegroup/gr916214277/?ref_=bo_ydw_table_9',
 'https://www.boxofficemojo.com/releasegroup/gr3499381253/?ref_=bo_ydw_table_10',
 'https://www.boxofficemojo.com/releasegroup/gr3197587973/?ref_=bo_ydw_table_11',
 'https://www.boxofficemojo.com/releasegroup/gr831869445/?ref_=bo_ydw_table_12',
 'https://www.boxofficemojo.com

In [112]:
service = Service(EdgeChromiumDriverManager().install())
def getMoviesInfo(links):
    movies_info = []
    for link in links:
        movies_info.append(getInfo(link))
    return movies_info

In [120]:
getMoviesInfo(info)

KeyboardInterrupt: 

In [42]:
data.shape

(201, 8)

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 11.9+ KB


In [44]:
data

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%,Year
0,1,Barbie,"$1,445,638,421","$636,238,421",44%,"$809,400,000",56%,2023
1,2,The Super Mario Bros. Movie,"$1,362,029,918","$574,934,330",42.2%,"$787,095,588",57.8%,2023
2,3,Oppenheimer,"$971,612,566","$329,862,540",34%,"$641,750,026",66%,2023
3,4,Guardians of the Galaxy Vol. 3,"$845,555,777","$358,995,815",42.5%,"$486,559,962",57.5%,2023
4,5,Fast X,"$704,875,015","$146,126,015",20.7%,"$558,749,000",79.3%,2023
...,...,...,...,...,...,...,...,...
196,197,Dream,"$8,317,251",-,-,"$8,317,251",100%,2023
197,198,Vacaciones de verano,"$8,209,854",-,-,"$8,209,854",100%,2023
198,199,On the Wandering Paths,"$8,186,415",-,-,"$8,186,415",100%,2023
199,200,Weekend Rebels,"$8,184,539",-,-,"$8,184,539",100%,2023


In [45]:
data['Year'].dtype

dtype('O')

In [46]:
data['Year'] = data['Year'].astype(str)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    object
dtypes: object(8)
memory usage: 12.7+ KB


In [49]:
data['Year'] = data['Year'].astype(int)

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 11.9+ KB
